In [1]:
# Copyright 2021 NVIDIA Corporation. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ================================

<img src="http://developer.download.nvidia.com/compute/machine-learning/frameworks/nvidia_logo.png" style="width: 90px; float: right;">

## Two-Stage Recommender Systems

In large scale recommender systems pipelines, the size of the item catalog (number of unique items) might be in the order of millions. At such scale, a typical setup is having two-stage pipeline, where a faster candidate retrieval model quickly extracts thousands of relevant items and a then a more powerful ranking model (i.e. with more features and more powerful architecture) ranks the top-k items that are going to be displayed to the user. For ML-based candidate retrieval model, as it needs to quickly score millions of items for a given user, a popular choices are models that can produce recommendation scores by just computing the dot product the user embeddings and item embeddings. Popular choices of such models are **Matrix Factorization**, which learns low-rank user and item embeddings, and the **Two-Tower architecture**, which is a neural network with two MLP towers where both user and item features are fed to generate user and item embeddings in the output.


### Dataset

In this notebook, we are building a Two-Tower model for Item Retrieval task using synthetic datasets that are mimicking the real [Ali-CCP: Alibaba Click and Conversion Prediction](https://tianchi.aliyun.com/dataset/dataDetail?dataId=408#1) dataset.
### Learning objectives
- Preparing the data with NVTabular
- Training and evaluating Two-Tower model with Merlin Models
- Exporting the model for deployment

### Importing Libraries

In [2]:
import os
os.environ["TF_GPU_ALLOCATOR"]="cuda_malloc_async"
import cudf

import nvtabular as nvt
from nvtabular.ops import *
from merlin.models.utils.example_utils import workflow_fit_transform

from merlin.schema.tags import Tags
from merlin.schema import Schema

import merlin.models.tf as mm
from merlin.io.dataset import Dataset

import tensorflow as tf

2022-04-05 01:38:59.433065: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-04-05 01:39:00.549719: I tensorflow/core/common_runtime/gpu/gpu_process_state.cc:214] Using CUDA malloc Async allocator for GPU: 0
2022-04-05 01:39:00.549854: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 16254 MB memory:  -> device: 0, name: Quadro GV100, pci bus id: 0000:15:00.0, compute capability: 7.0


In [3]:
# disable INFO and DEBUG logging everywhere
import logging
logging.disable(logging.WARNING)

### Feature Engineering with NVTabular

Let's generate synthetic train and validation dataset objects.

In [4]:
from merlin.models.data.synthetic import generate_data

DATA_FOLDER = os.environ.get("DATA_FOLDER", "/workspace/data/")

train = generate_data("aliccp-raw", 1000000)
valid = generate_data("aliccp-raw", 1000000)

# save the datasets as parquet files
train.to_ddf().to_parquet(os.path.join(DATA_FOLDER, 'train'))
valid.to_ddf().to_parquet(os.path.join(DATA_FOLDER, 'valid'))

(None,)

In [7]:
# define paths for the raw parquet files
train_path = os.path.join(DATA_FOLDER, "train", "part.0.parquet")
valid_path = os.path.join(DATA_FOLDER, "valid", "part.0.parquet")
output_path =os.path.join(DATA_FOLDER, "processed")

We keep only positive interactions where clicks==1 in the dataset with `Filter()` op.

In [8]:
user_id = ["user_id"] >> Categorify() >> TagAsUserID()
item_id = ["item_id"] >> Categorify() >> TagAsItemID()

item_features = ["item_category", "item_shop", "item_brand"] >> Categorify() >> TagAsItemFeatures()

user_features = ['user_shops', 'user_profile', 'user_group', 
       'user_gender', 'user_age', 'user_consumption_2', 'user_is_occupied',
       'user_geography', 'user_intentions', 'user_brands', 'user_categories'] \
        >> Categorify() >> TagAsUserFeatures()

inputs = user_id + item_id + item_features + user_features + ['click'] 

outputs = inputs >> Filter(f=lambda df: df["click"] == 1)

workflow_fit_transform(outputs, train_path, valid_path, output_path)

/usr/local/lib/python3.8/dist-packages/cudf/core/dataframe.py:1253: UserWarning: The deep parameter is ignored and is only included for pandas compatibility.
  warnings.warn(


## Building a Two-Tower Model with Merlin Models

We will use Two-Tower Model for item retrieval task. Real-world large scale recommender systems have hundreds of millions of items (products) and users. Thus, these systems often composed of two stages: candidate generation (retrieval) and ranking (scoring the retrieved items). At candidate generation step, a subset of relevant items from large item corpus is retrieved. You can read more about two stage Recommender Systems here. In this example, we're going to focus on the retrieval stage.

A Two-Tower Model consists of item (candidate) and user (query) encoder towers. With two towers, the model can learn representations (embeddings) for queries and candidates separately. 

<img src="./images/TwoTower.png"  width="30%">

Image Adapted from: [Off-policy Learning in Two-stage Recommender Systems](https://dl.acm.org/doi/abs/10.1145/3366423.3380130)

We use the `schema` object to define our model.

In [9]:
train = Dataset(os.path.join(output_path, 'train', '*.parquet'))
valid = Dataset(os.path.join(output_path, 'valid', '*.parquet'))

schema = train.schema

In [10]:
schema = schema.select_by_tag([Tags.ITEM_ID, Tags.USER_ID, Tags.ITEM, Tags.USER])

We can print out the feature column names.

In [11]:
schema.column_names

['user_id',
 'item_id',
 'item_category',
 'item_shop',
 'item_brand',
 'user_shops',
 'user_profile',
 'user_group',
 'user_gender',
 'user_age',
 'user_consumption_2',
 'user_is_occupied',
 'user_geography',
 'user_intentions',
 'user_brands',
 'user_categories']

We expect the label names to be empty.

In [12]:
label_names = schema.select_by_tag(Tags.TARGET).column_names
label_names

[]

### Negative sampling
Many datasets for recommender systems contain implicit feedback with logs of user interactions like clicks, add-to-cart, purchases, music listening events, rather than explicit ratings that reflects user preferences over items. To be able to learn from implicit feedback, we use the general (and naive) assumption that the interacted items are more relevant for the user than the non-interacted ones.
In Merlin Models we provide some scalable negative sampling algorithms for the Item Retrieval Task. In particular, we use in this example the in-batch sampling algorithm which uses the items interacted by other users as negatives within the same mini-batch.

### Building the Model

Now, let's build our Two-Tower model. In a nutshell, we aggregate all user features to feed in user tower and feed the item features to the item tower. Then we compute the positive score by multiplying the user embedding with the item embedding and sample negative items (read more about negative sampling [here](https://openreview.net/pdf?id=824xC-SgWgU) and [here](https://medium.com/mlearning-ai/overview-negative-sampling-on-recommendation-systems-230a051c6cd7)), whose item embeddings are also multiplied by the user embedding. Then we apply the loss function on top of the positive and negative scores.

In [13]:
model = mm.TwoTowerModel(
    schema,
    query_tower=mm.MLPBlock([128, 64], no_activation_last_layer=True),        
    loss="categorical_crossentropy",  
    samplers=[mm.InBatchSampler()],
    embedding_options = mm.EmbeddingOptions(infer_embedding_sizes=True),
    metrics=[mm.RecallAt(10), mm.NDCGAt(10)]
)

Let's explain the parameters in the TwoTowerModel():
- no_activation_last_layer: when set True, no activation is used for top hidden layer. Learn more [here](https://storage.googleapis.com/pub-tools-public-publication-data/pdf/b9f4e78a8830fe5afcf2f0452862fb3c0d6584ea.pdf).
- infer_embedding_sizes: when set True, automatically defines the embedding dimension from the feature cardinality in the schema

**Metrics:**

The following information retrieval metrics are used to compute the Top-10 accuracy of recommendation lists containing all items:

- **Normalized Discounted Cumulative Gain (NDCG@10)**: NDCG accounts for rank of the relevant item in the recommendation list and is a more fine-grained metric than HR, which only verifies whether the relevant item is among the top-k items.

- **Recall@10**: Also known as HitRate@n when there is only one relevant item in the recommendation list. Recall just verifies whether the relevant item is among the top-n items.

We need to initialize the dataloaders.

In [14]:
model.set_retrieval_candidates_for_evaluation(train)

opt = tf.keras.optimizers.Adagrad(learning_rate=0.003)
model.compile(optimizer=opt, run_eagerly=False)
model.fit(train, validation_data=valid, batch_size=4096, epochs=3)

2022-04-05 01:39:29.887002: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


Epoch 1/3
121/123 [============================>.] - ETA: 0s - recall_at_10: 0.0055 - ndcg_10: 0.0040 - loss: 8.3060 - regularization_loss: 0.0000e+00 - total_loss: 8.3060

2022-04-05 01:39:46.143709: W tensorflow/core/grappler/optimizers/loop_optimizer.cc:907] Skipping loop optimization for Merge node with control input: cond/then/_0/cond/cond/branch_executed/_185


123/123 [==============================] - 17s 90ms/step - recall_at_10: 0.0055 - ndcg_10: 0.0040 - loss: 8.2864 - regularization_loss: 0.0000e+00 - total_loss: 8.2864 - val_recall_at_10: 0.0044 - val_ndcg_10: 0.0021 - val_loss: 8.2752 - val_regularization_loss: 0.0000e+00 - val_total_loss: 8.2752
Epoch 2/3
123/123 [==============================] - 9s 69ms/step - recall_at_10: 0.0047 - ndcg_10: 0.0034 - loss: 8.2853 - regularization_loss: 0.0000e+00 - total_loss: 8.2853 - val_recall_at_10: 0.0048 - val_ndcg_10: 0.0022 - val_loss: 8.2750 - val_regularization_loss: 0.0000e+00 - val_total_loss: 8.2750
Epoch 3/3
123/123 [==============================] - 9s 73ms/step - recall_at_10: 0.0047 - ndcg_10: 0.0033 - loss: 8.2846 - regularization_loss: 0.0000e+00 - total_loss: 8.2846 - val_recall_at_10: 0.0048 - val_ndcg_10: 0.0022 - val_loss: 8.2749 - val_regularization_loss: 0.0000e+00 - val_total_loss: 8.2749


## Exporting Retrieval Models

So far we have trained and evaluated our Retrieval model. Now, the next step is to deploy our model and generate top-K recommendations given a user (query). We can efficiently serve our model by indexing the trained item embeddings into an **Approximate Nearest Neighbors (ANN)** engine. Basically, for a given user query vector, that is generated passing the user features into user tower of retrieval model, we do an ANN search query to find the ids of nearby item vectors, and at serve time, we score user embeddings over all indexed top-K item embeddings within the ANN engine.

In doing so, we need to export
 
- user (query) tower
- item and user features
- item embeddings

#### Save User (query) tower

We are able to save the user tower model as a TF model to disk. The user tower model is needed to generate a user embedding vector when a user feature vector <i>x</i> is fed into that model.

In [16]:
query_tower = model.retrieval_block.query_block()
query_tower.save('query_tower')

#### Extract and save User features

With `unique_rows_by_features` utility function we can easily extract both unique user and item features tables as cuDF dataframes. Note that for user features table, we use `USER` and `USER_ID` tags.

In [17]:
from merlin.models.utils.dataset import unique_rows_by_features
user_features = unique_rows_by_features(train, Tags.USER, Tags.USER_ID).compute().reset_index(drop=True)

In [18]:
user_features.head()

,user_id,user_shops,user_profile,user_group,user_gender,user_age,user_consumption_2,user_is_occupied,user_geography,user_intentions,user_brands,user_categories
0,1,1,1,1,1,1,1,1,1,1,1,1
1,2,2,1,1,1,1,1,1,1,2,2,2
2,3,3,1,1,1,1,1,1,1,3,3,3
3,4,4,1,1,1,1,1,1,1,4,4,4
4,5,5,1,1,1,1,1,1,1,5,5,5


In [19]:
user_features.shape

(725, 12)

In [20]:
# save to disk
user_features.to_parquet('user_features.parquet')

#### Extract and save Item features

In [21]:
item_features = unique_rows_by_features(train, Tags.ITEM, Tags.ITEM_ID).compute().reset_index(drop=True)

In [22]:
item_features.head()

,item_id,item_category,item_shop,item_brand
0,1,1,1,1
1,2,2,2,2
2,3,3,3,3
3,4,4,4,4
4,5,5,5,5


In [23]:
# save to disk
item_features.to_parquet('item_features.parquet')

#### Extract and save Item embeddings

In [24]:
item_embs = model.item_embeddings(Dataset(item_features, schema=schema), batch_size=1024)
item_embs_df = item_embs.compute(scheduler="synchronous")

In [25]:
item_embs_df

,item_id,item_category,item_shop,item_brand,0,1,2,3,4,5,...,54,55,56,57,58,59,60,61,62,63
0,1,1,1,1,-0.080138,0.164991,-0.088390,-0.084368,-0.170704,-0.194693,...,-0.121886,-0.160051,0.239783,0.190130,0.294039,0.173054,-0.091124,0.053051,-0.061462,0.001636
1,2,2,2,2,0.065198,0.008223,-0.026139,0.027760,0.026469,-0.109828,...,0.008733,0.114419,-0.173410,0.064015,0.073164,0.176540,-0.061230,0.010753,0.053984,-0.101455
2,3,3,3,3,0.169136,-0.035362,-0.027654,-0.284353,-0.017567,-0.139145,...,-0.092729,-0.018937,-0.090321,0.180357,-0.110631,0.102139,0.148058,0.170666,0.267043,-0.112293
3,4,4,4,4,0.037014,-0.062965,-0.050965,-0.014348,0.046101,-0.081345,...,0.066225,-0.069351,-0.109032,0.094101,0.144096,0.140754,-0.050377,-0.034933,-0.107016,-0.070730
4,5,5,5,5,-0.248760,0.086720,-0.268344,-0.252266,-0.225200,-0.004744,...,0.099194,-0.215265,0.187329,0.281703,-0.058103,0.114930,0.012834,-0.011960,0.095858,-0.030840
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
712,832,832,832,832,-0.037588,0.013589,-0.106824,0.052419,-0.213757,-0.084607,...,0.049071,0.032056,-0.157333,0.045341,-0.004577,0.090677,0.000008,-0.018588,0.047204,-0.206918
713,833,833,833,833,-0.040116,0.084456,-0.076909,-0.302175,-0.031607,-0.291037,...,-0.191154,-0.099356,0.167159,0.195780,0.089412,0.183644,-0.032043,0.196099,0.203973,-0.063994
714,835,835,835,835,-0.036690,0.196705,-0.051777,-0.022180,-0.044232,-0.145672,...,0.100373,0.045667,-0.038770,0.013661,-0.031325,-0.074642,-0.047355,-0.111672,0.076764,-0.086005
715,838,838,838,838,-0.068956,0.110597,-0.170252,-0.230961,0.004850,-0.214517,...,-0.043954,-0.141380,-0.073150,0.262040,-0.020642,0.301193,-0.005274,-0.043695,0.093276,-0.091933


In [26]:
# select only embedding columns
item_embeddings = item_embs_df.iloc[:, 4:]

In [27]:
item_embeddings.head()

,0,1,2,3,4,5,6,7,8,9,...,54,55,56,57,58,59,60,61,62,63
0,-0.080138,0.164991,-0.088390,-0.084368,-0.170704,-0.194693,0.028913,-0.058403,-0.246632,0.034682,...,-0.121886,-0.160051,0.239783,0.190130,0.294039,0.173054,-0.091124,0.053051,-0.061462,0.001636
1,0.065198,0.008223,-0.026139,0.027760,0.026469,-0.109828,0.081742,0.146817,-0.034586,-0.161586,...,0.008733,0.114419,-0.173410,0.064015,0.073164,0.176540,-0.061230,0.010753,0.053984,-0.101455
2,0.169136,-0.035362,-0.027654,-0.284353,-0.017567,-0.139145,0.066012,0.218829,-0.029755,-0.065688,...,-0.092729,-0.018937,-0.090321,0.180357,-0.110631,0.102139,0.148058,0.170666,0.267043,-0.112293
3,0.037014,-0.062965,-0.050965,-0.014348,0.046101,-0.081345,-0.232820,-0.091644,0.239366,-0.126221,...,0.066225,-0.069351,-0.109032,0.094101,0.144096,0.140754,-0.050377,-0.034933,-0.107016,-0.070730
4,-0.248760,0.086720,-0.268344,-0.252266,-0.225200,-0.004744,0.036384,-0.143048,-0.153263,-0.088398,...,0.099194,-0.215265,0.187329,0.281703,-0.058103,0.114930,0.012834,-0.011960,0.095858,-0.030840


In [28]:
# save to disk
item_embeddings.to_parquet('item_embeddings.parquet')

That's it. You have learned how to train and evaluate your Two-Tower retrieval model, and then how to export the required components to be able to deploy this model to generate recommendations. In order to learn more on serving a model to [Triton Inference Server](https://github.com/triton-inference-server/server), please explore the examples in the [Merlin](https://github.com/NVIDIA-Merlin/Merlin) and [Merlin Systems](https://github.com/NVIDIA-Merlin/systems) repos.